In [1]:
import torch
import torch.optim as optim
import os
from load_data_set import get_data_loaders
from generator import GENERATOR
from discriminator import Discriminator
from lossFunctions import discriminator_gan_loss, generator_gan_loss, compute_yolo_loss
import numpy as np
from ultralytics import YOLO


In [ ]:
#download the dataset if it is not already downloaded
!curl -L "https://universe.roboflow.com/ds/glquRaJDf9?key=1kWinBXVTQ" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [4]:
# Training parameters
EPOCHS = 1
BATCH_SIZE = 32
IMAGE_SIZE = 256
LATENT_DIM = 100  # Size of the noise vector (for the generator)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SAVE_DIR = "./gan_checkpoints"

# Create save directory if it doesn't exist
os.makedirs(SAVE_DIR, exist_ok=True)

In [6]:
# Define your directories as arguments
train_dir = "/home/salma/graduation_project/YOLO/yolov8/Self-Driving Cars.v1i.yolov8/train/images"
valid_dir = "/home/salma/graduation_project/YOLO/yolov8/Self-Driving Cars.v1i.yolov8/valid/images"
test_dir = "/home/salma/graduation_project/YOLO/yolov8/Self-Driving Cars.v1i.yolov8/test/images"

# Load dataset
dataloaders = get_data_loaders(train_dir, valid_dir, test_dir, batch_size=16, image_size=256)


# Initialize models
image_height, image_width, n_channels = 256, 256, 3
generator = GENERATOR(in_channels=3, out_channels=3).to(DEVICE)
discriminator = Discriminator(image_height, image_width, n_channels).to(DEVICE)
yolo_model = YOLO("../yolov8n_TrafficSigns.pt")  # Load the YOLOv8 model